In [1]:
!pip install numpy sentence-transformers bertopic hdbscan nltk scann
import nltk
nltk.download('punkt')
import nltk
nltk.download('punkt_tab')
!pip install sentence-transformers bertopic hdbscan umap-learn scann nltk datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
# # === IMPORTS ===
# import os
# import random
# import numpy as np
# import torch
# import nltk
# import logging

# from collections import defaultdict
# from nltk.tokenize import sent_tokenize
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic
# from bertopic.representation import KeyBERTInspired
# from hdbscan import HDBSCAN
# from umap import UMAP
# import scann

# # === ENVIRONMENT SETUP ===
# SEED = 42
# np.random.seed(SEED)
# random.seed(SEED)
# os.environ["PYTHONHASHSEED"] = str(SEED)
# torch.manual_seed(SEED)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(SEED)
# torch.use_deterministic_algorithms(True)
# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)
# nltk.download("punkt")


# # === CLASS FOR TOPIC SEARCHER ===
# class AllergyTopicSearcher:
#     def __init__(self, chunks, entities_per_chunk, umap_params, hdbscan_params, model_name="all-MiniLM-L6-v2"):
#         self.chunks = chunks
#         self.entities_per_chunk = entities_per_chunk
#         self.embedding_model = SentenceTransformer(model_name)

#         self.umap_params = umap_params
#         self.hdbscan_params = hdbscan_params

#         self.topic_model = None
#         self.topic_metadata = []
#         self.topic_embeddings = None
#         self.searcher = None

#         self._prepare()

#     def _prepare(self):
#         entity_context_pairs = []

#         for idx, ents in enumerate(self.entities_per_chunk):
#             chunk = self.chunks[idx].lower()
#             sentences = sent_tokenize(chunk)
#             for ent in ents:
#                 ent_lower = ent.lower()
#                 for sent in sentences:
#                     if ent_lower in sent:
#                         entity_context_pairs.append((ent_lower, sent.strip()))
#                         break

#         if not entity_context_pairs:
#             raise ValueError("No entity-context pairs extracted!")

#         # Embed entity-contexts
#         contextual_texts = [f"{ent}: {context}" for ent, context in entity_context_pairs]
#         contextual_embeddings = self.embedding_model.encode(contextual_texts, normalize_embeddings=True)

#         # Topic Modeling
#         umap_model = UMAP(**self.umap_params)
#         hdbscan_model = HDBSCAN(**self.hdbscan_params, prediction_data=True)

#         self.topic_model = BERTopic(
#             embedding_model=self.embedding_model,
#             umap_model=umap_model,
#             hdbscan_model=hdbscan_model,
#             representation_model=KeyBERTInspired(),
#             calculate_probabilities=True,
#             verbose=False,
#         )

#         topics, _ = self.topic_model.fit_transform(contextual_texts, embeddings=contextual_embeddings)

#         # Metadata aggregation
#         topic_to_contexts = defaultdict(list)
#         topic_to_entities = defaultdict(set)
#         topic_to_embeddings = defaultdict(list)

#         for i, topic in enumerate(topics):
#             ent, context = entity_context_pairs[i]
#             topic_to_contexts[topic].append(context)
#             topic_to_entities[topic].add(ent)
#             topic_to_embeddings[topic].append(contextual_embeddings[i])

#         topic_embeddings = []
#         topic_metadata = []

#         for topic_id in topic_to_contexts:
#             embeddings = topic_to_embeddings[topic_id]
#             mean_emb = np.mean(embeddings, axis=0)
#             mean_emb /= np.linalg.norm(mean_emb) + 1e-10
#             topic_embeddings.append(mean_emb)
#             topic_metadata.append({
#                 "topic_id": topic_id,
#                 "entities": list(topic_to_entities[topic_id]),
#                 "sentences": topic_to_contexts[topic_id],
#                 "sentence_embeddings": np.array(embeddings)
#             })

#         self.topic_embeddings = np.array(topic_embeddings)
#         self.topic_metadata = topic_metadata

#         if len(self.topic_embeddings) < 1:
#             raise RuntimeError("No topic embeddings to index.")

#         num_clusters = min(len(self.topic_embeddings), 3)
#         self.searcher = (
#             scann.scann_ops_pybind.builder(self.topic_embeddings, 3, "dot_product")
#             .tree(num_leaves=num_clusters, num_leaves_to_search=2, training_sample_size=len(self.topic_embeddings))
#             .score_brute_force()
#             .reorder(3)
#             .build()
#         )

#     def search(self, query, top_k_topics=1, top_k_sents=1):
#         query_emb = self.embedding_model.encode([query], normalize_embeddings=True)[0]
#         neighbors, scores = self.searcher.search(query_emb, final_num_neighbors=top_k_topics)

#         results = []
#         for idx in neighbors:
#             meta = self.topic_metadata[idx]
#             seen = set()
#             unique_sentences = []
#             unique_embeddings = []

#             for sent, emb in zip(meta["sentences"], meta["sentence_embeddings"]):
#                 if sent not in seen:
#                     seen.add(sent)
#                     unique_sentences.append(sent)
#                     unique_embeddings.append(emb)

#             sent_embs = np.array(unique_embeddings)
#             sent_embs_norm = sent_embs / np.linalg.norm(sent_embs, axis=1, keepdims=True)
#             sims = np.dot(sent_embs_norm, query_emb)
#             top_indices = sims.argsort()[::-1][:top_k_sents]
#             top_sents = [(unique_sentences[i], sims[i]) for i in top_indices]

#             results.append({
#                 "topic_id": meta["topic_id"],
#                 "entities": meta["entities"],
#                 "sentences": top_sents,
#             })

#         return results


# # === DATASET ===
# allergy_dataset = {
#     "name": "Allergy Dataset",
#     "chunks": [
#         "Patient has peanut allergy causing hives and swelling. Anaphylaxis noted once during a reaction.",
#         "Allergic rhinitis, or hay fever, results from exposure to pollen, dust, or pet dander.",
#         "Severe anaphylaxis symptoms require immediate treatment with epinephrine.",
#         "Food allergies to milk and eggs can cause skin reactions like urticaria and eczema.",
#         "Cold weather does not cause allergy symptoms in this patient."
#     ],
#     "entities": [
#         ["peanut allergy", "hives", "swelling", "anaphylaxis"],
#         ["allergic rhinitis", "hay fever", "pollen", "dust", "pet dander"],
#         ["anaphylaxis", "epinephrine", "treatment"],
#         ["food allergies", "milk", "eggs", "urticaria", "eczema"],
#         ["cold weather", "allergy symptoms"]
#     ]
# }

# # === BEST PARAMETERS ===
# best_umap = {"n_neighbors": 5, "n_components": 5, "min_dist": 0.1, "metric": "cosine"}
# best_hdbscan = {"min_cluster_size": 2, "min_samples": 1, "metric": "euclidean"}

# # === INITIALIZE SEARCHER ===
# print("Preparing Allergy Topic Searcher...")
# searcher = AllergyTopicSearcher(
#     chunks=allergy_dataset["chunks"],
#     entities_per_chunk=allergy_dataset["entities"],
#     umap_params=best_umap,
#     hdbscan_params=best_hdbscan,
# )
# print("✅ Model ready for querying.")

# # === QUERY LOOP ===
# print("\n=== Allergy Topic Search ===")
# while True:
#     query = input("\nEnter a query (or type 'exit' to quit): ").strip()
#     if query.lower() in {"exit", "quit"}:
#         print("Goodbye!")
#         break

#     results = searcher.search(query, top_k_topics=1, top_k_sents=3)
#     print(f"\n🔎 Top results for: '{query}'")
#     for res in results:
#         print(f"🧠 Topic ID: {res['topic_id']}")
#         print(f"🔗 Related Entities: {', '.join(res['entities'])}")
#         for sent, _ in res["sentences"]:
#             print(f"✓ {sent}")


In [4]:
# === IMPORTS ===
import os
import random
import numpy as np
import torch
import nltk
import logging

from collections import defaultdict
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from umap import UMAP
import scann

# === ENVIRONMENT SETUP ===
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
nltk.download("punkt")


# === CLASS FOR TOPIC SEARCHER ===
class AllergyTopicSearcher:
    def __init__(self, chunks, entities_per_chunk, umap_params, hdbscan_params, model_name="all-MiniLM-L6-v2"):
        self.chunks = chunks
        self.entities_per_chunk = entities_per_chunk
        self.embedding_model = SentenceTransformer(model_name)

        self.umap_params = umap_params
        self.hdbscan_params = hdbscan_params

        self.topic_model = None
        self.topic_metadata = []
        self.topic_embeddings = None
        self.searcher = None

        self._prepare()

    def _prepare(self):
        entity_context_pairs = []

        for idx, ents in enumerate(self.entities_per_chunk):
            chunk = self.chunks[idx].lower()
            sentences = sent_tokenize(chunk)
            for ent in ents:
                ent_lower = ent.lower()
                for sent in sentences:
                    if ent_lower in sent:
                        entity_context_pairs.append((ent_lower, sent.strip()))
                        break

        if not entity_context_pairs:
            raise ValueError("No entity-context pairs extracted!")

        # Embed entity-contexts
        contextual_texts = [f"{ent}: {context}" for ent, context in entity_context_pairs]
        contextual_embeddings = self.embedding_model.encode(contextual_texts, normalize_embeddings=True)

        # Topic Modeling
        umap_model = UMAP(**self.umap_params)
        hdbscan_model = HDBSCAN(**self.hdbscan_params, prediction_data=True)

        self.topic_model = BERTopic(
            embedding_model=self.embedding_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            representation_model=KeyBERTInspired(),
            calculate_probabilities=True,
            verbose=False,
        )

        topics, _ = self.topic_model.fit_transform(contextual_texts, embeddings=contextual_embeddings)

        # Metadata aggregation
        topic_to_contexts = defaultdict(list)
        topic_to_entities = defaultdict(set)
        topic_to_embeddings = defaultdict(list)

        for i, topic in enumerate(topics):
            ent, context = entity_context_pairs[i]
            topic_to_contexts[topic].append(context)
            topic_to_entities[topic].add(ent)
            topic_to_embeddings[topic].append(contextual_embeddings[i])

        topic_embeddings = []
        topic_metadata = []

        for topic_id in topic_to_contexts:
            embeddings = topic_to_embeddings[topic_id]
            mean_emb = np.mean(embeddings, axis=0)
            mean_emb /= np.linalg.norm(mean_emb) + 1e-10
            topic_embeddings.append(mean_emb)
            topic_metadata.append({
                "topic_id": topic_id,
                "entities": list(topic_to_entities[topic_id]),
                "sentences": topic_to_contexts[topic_id],
                "sentence_embeddings": np.array(embeddings)
            })

        self.topic_embeddings = np.array(topic_embeddings)
        self.topic_metadata = topic_metadata

        # === Added print: topics and their entities ===
        print("\n=== Topics and Associated Entities ===")
        for meta in self.topic_metadata:
            print(f"Topic ID: {meta['topic_id']}, Entities: {', '.join(meta['entities'])}")

        if len(self.topic_embeddings) < 1:
            raise RuntimeError("No topic embeddings to index.")

        num_clusters = min(len(self.topic_embeddings), 3)
        self.searcher = (
            scann.scann_ops_pybind.builder(self.topic_embeddings, 3, "dot_product")
            .tree(num_leaves=num_clusters, num_leaves_to_search=2, training_sample_size=len(self.topic_embeddings))
            .score_brute_force()
            .reorder(3)
            .build()
        )

    def search(self, query, top_k_topics=1, top_k_sents=1):
        query_emb = self.embedding_model.encode([query], normalize_embeddings=True)[0]
        neighbors, scores = self.searcher.search(query_emb, final_num_neighbors=top_k_topics)

        results = []
        for idx in neighbors:
            meta = self.topic_metadata[idx]
            seen = set()
            unique_sentences = []
            unique_embeddings = []

            for sent, emb in zip(meta["sentences"], meta["sentence_embeddings"]):
                if sent not in seen:
                    seen.add(sent)
                    unique_sentences.append(sent)
                    unique_embeddings.append(emb)

            sent_embs = np.array(unique_embeddings)
            sent_embs_norm = sent_embs / np.linalg.norm(sent_embs, axis=1, keepdims=True)
            sims = np.dot(sent_embs_norm, query_emb)
            top_indices = sims.argsort()[::-1][:top_k_sents]
            top_sents = [(unique_sentences[i], sims[i]) for i in top_indices]

            results.append({
                "topic_id": meta["topic_id"],
                "entities": meta["entities"],
                "sentences": top_sents,
            })

        return results


# === DATASET ===
allergy_dataset = {
    "name": "Allergy Dataset",
    "chunks": [
        "Patient has peanut allergy causing hives and swelling. Anaphylaxis noted once during a reaction.",
        "Allergic rhinitis, or hay fever, results from exposure to pollen, dust, or pet dander.",
        "Severe anaphylaxis symptoms require immediate treatment with epinephrine.",
        "Food allergies to milk and eggs can cause skin reactions like urticaria and eczema.",
        "Cold weather does not cause allergy symptoms in this patient."
    ],
    "entities": [
        ["peanut allergy", "hives", "swelling", "anaphylaxis"],
        ["allergic rhinitis", "hay fever", "pollen", "dust", "pet dander"],
        ["anaphylaxis", "epinephrine", "treatment"],
        ["food allergies", "milk", "eggs", "urticaria", "eczema"],
        ["cold weather", "allergy symptoms"]
    ]
}

# === BEST PARAMETERS ===
best_umap = {"n_neighbors": 5, "n_components": 5, "min_dist": 0.1, "metric": "cosine"}
best_hdbscan = {"min_cluster_size": 2, "min_samples": 1, "metric": "euclidean"}

# === INITIALIZE SEARCHER ===
print("Preparing Allergy Topic Searcher...")
searcher = AllergyTopicSearcher(
    chunks=allergy_dataset["chunks"],
    entities_per_chunk=allergy_dataset["entities"],
    umap_params=best_umap,
    hdbscan_params=best_hdbscan,
)
print("✅ Model ready for querying.")

# === QUERY LOOP ===
print("\n=== Allergy Topic Search ===")
while True:
    query = input("\nEnter a query (or type 'exit' to quit): ").strip()
    if query.lower() in {"exit", "quit"}:
        print("Goodbye!")
        break

    results = searcher.search(query, top_k_topics=1, top_k_sents=3)
    print(f"\n🔎 Top results for: '{query}'")
    for res in results:
        print(f"🧠 Topic ID: {res['topic_id']}")
        print(f"🔗 Related Entities: {', '.join(res['entities'])}")
        for sent, _ in res["sentences"]:
            print(f"✓ {sent}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Preparing Allergy Topic Searcher...

=== Topics and Associated Entities ===
Topic ID: 3, Entities: swelling, peanut allergy, hives
Topic ID: 2, Entities: treatment, anaphylaxis, epinephrine
Topic ID: 0, Entities: allergic rhinitis, pollen, dust, allergy symptoms, cold weather, pet dander, hay fever
Topic ID: 1, Entities: eggs, eczema, food allergies, urticaria, milk
✅ Model ready for querying.

=== Allergy Topic Search ===

Enter a query (or type 'exit' to quit): food

🔎 Top results for: 'food'
🧠 Topic ID: 1
🔗 Related Entities: eggs, eczema, food allergies, urticaria, milk
✓ food allergies to milk and eggs can cause skin reactions like urticaria and eczema.

Enter a query (or type 'exit' to quit): symptoms 

🔎 Top results for: 'symptoms'
🧠 Topic ID: 0
🔗 Related Entities: allergic rhinitis, pollen, dust, allergy symptoms, cold weather, pet dander, hay fever
✓ cold weather does not cause allergy symptoms in this patient.
✓ allergic rhinitis, or hay fever, results from exposure to pollen,